# High-level Keras (CNTK) Example

In [1]:
import os
import sys
import numpy as np
os.environ['KERAS_BACKEND'] = "cntk"
import keras as K
import cntk
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from common.params import *
from common.utils import *

Using CNTK backend


In [2]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Keras: ", K.__version__)
print("Numpy: ", np.__version__)
print("CNTK: ", cntk.__version__)
print(K.backend.backend())
print(K.backend.image_data_format())
print("GPU: ", get_gpu_name())

OS:  linux
Python:  3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Keras:  2.0.6
Numpy:  1.13.1
CNTK:  2.1
cntk
channels_last


In [3]:
def create_symbol():
    model = Sequential()
    
    model.add(Conv2D(50, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)))
    model.add(Conv2D(50, kernel_size=(3, 3), padding='same', activation='relu'))    
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(100, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(100, kernel_size=(3, 3), padding='same', activation='relu'))    
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))
        
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(N_CLASSES, activation='softmax'))
    return model

In [4]:
def init_model(m):
    m.compile(
        loss = "categorical_crossentropy",
        optimizer = K.optimizers.SGD(LR, MOMENTUM),
        metrics = ['accuracy'])
    return m

In [5]:
%%time
# Data into format for library
#x_train, x_test, y_train, y_test = mnist_for_library(channel_first=False, one_hot=True)
x_train, x_test, y_train, y_test = cifar_for_library(channel_first=False, one_hot=True)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Done.
Extracting files...
Done.
Preparing train set...
Preparing test set...
Done.
(50000, 32, 32, 3) (10000, 32, 32, 3) (50000, 10) (10000, 10)
float32 float32 int32 int32
CPU times: user 3 s, sys: 1.59 s, total: 4.59 s
Wall time: 32 s


In [6]:
%%time
# Load symbol
sym = create_symbol()

CPU times: user 188 ms, sys: 220 ms, total: 408 ms
Wall time: 417 ms


In [7]:
%%time
# Initialise model
model = init_model(sym)

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 7.49 ms


In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 50)        1400      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 50)        22550     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 50)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 50)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 100)       45100     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 100)       90100     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 100)         0         
__________

In [9]:
%%time
# Train model
model.fit(x_train,
          y_train,
          batch_size=BATCHSIZE,
          epochs=EPOCHS,
          verbose=1)

Epoch 1/10


/anaconda/envs/py35/lib/python3.5/site-packages/cntk/core.py:82: RuntimeWarning: data is not C contiguous; rearrange your data/computation to avoid costly data conversions
  RuntimeWarning)


50000/50000 [==============================] - 53s - loss: 1.8506 - acc: 0.3231    
Epoch 2/10
50000/50000 [==============================] - 58s - loss: 1.4362 - acc: 0.4817    
Epoch 3/10
50000/50000 [==============================] - 60s - loss: 1.2332 - acc: 0.5589    
Epoch 4/10
50000/50000 [==============================] - 59s - loss: 1.0640 - acc: 0.6224    
Epoch 5/10
50000/50000 [==============================] - 59s - loss: 0.9360 - acc: 0.6667    
Epoch 6/10
50000/50000 [==============================] - 59s - loss: 0.8388 - acc: 0.7058    
Epoch 7/10
50000/50000 [==============================] - 59s - loss: 0.7628 - acc: 0.7315    
Epoch 8/10
50000/50000 [==============================] - 59s - loss: 0.6945 - acc: 0.7542    
Epoch 9/10
50000/50000 [==============================] - 60s - loss: 0.6380 - acc: 0.7763    
Epoch 10/10
50000/50000 [==============================] - 58s - loss: 0.5796 - acc: 0.7946    
CPU times: user 5min 33s, sys: 1min 28s, total: 7min 1s
Wall

In [10]:
%%time
y_guess = model.predict(x_test, batch_size=BATCHSIZE)
y_guess = np.argmax(y_guess, axis=-1)
y_truth = np.argmax(y_test, axis=-1)

/anaconda/envs/py35/lib/python3.5/site-packages/cntk/core.py:82: RuntimeWarning: data is not C contiguous; rearrange your data/computation to avoid costly data conversions
  RuntimeWarning)


CPU times: user 2.3 s, sys: 652 ms, total: 2.96 s
Wall time: 4.12 s


In [11]:
print("Accuracy: ", sum(y_guess == y_truth)/len(y_guess))

Accuracy:  0.7643
